In [1]:
import os
import json

import numpy as np
import pandas as pd

import fasttext
import fasttext.util

import tensorflow as tf
import tensorflow_text

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, train_test_split

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_recall_fscore_support, classification_report, precision_recall_curve

import spacy

import warnings
warnings.filterwarnings('ignore')

In [3]:
PRETRAINED_FASTTEXT_MODEL = os.getcwd() + '/../../models/lid.176.bin'
LANG_EN = "__label__en"
LANG_DE = "__label__de"

fasttext.FastText.eprint = lambda x: None
fasttext_model = fasttext.load_model(PRETRAINED_FASTTEXT_MODEL)

def detect_language(text):
    
    lang_label = fasttext_model.predict(text)[0][0].split('__label__')[1]
    return lang_label

In [229]:
text = """미소와 함께 '일상의 변화'를 만들 '미소 메이커스'를 찾습니다  미소는 국내 No.1 O2O 홈서비스 플랫폼 기업입니다. "Hotel-like service in your home"  이라는 비전 아래, 고객에게 더욱 행복한 경험을 더욱 많이 제공하고자 합니다. 대표 서비스인 '홈클리닝'을 중심으로, '이사'/'가전청소'/'인테리어'/'펫시팅' 등  70여가지의 서비스 로 사업 범위를 확장하였습니다. 고객 만족 원칙과 데이터 기반 기술로 매년 2배가량 돋보이게 성장하고 있습니다. 2021년 현재 Series A 단계이며, 총 투자규모는 약 130억 원입니다. 한국 O2O 기업으로는 최초로 실리콘밸리의 최대 벤처 투자사 ‘Y Combinator’로부터 31억 원의 투자를 유치했습니다. 누적 매출액은  1,000억 원 , 누적 주문건수  300만 건  및 누적 파트너수  40,000명 을 돌파했습니다.  미소의 일 하는 방식 엿보기 Work hard on the Right Things 미소는 올바른 일에 집중합니다. 미소 팀블로그에서 미소 메이커스의 이야기를 들어 보세요!    예비 미소 메이커스를 위한 참고 사이트   채용과 관련한 모든 문의사항은,  recruit@getmiso.com (People Team) 으로 부탁드립니다."""
fasttext_model.predict(text)[0][0].split('__label__')[1]

'ko'

In [2]:
tf.saved_model.LoadOptions(
    allow_partial_checkpoint=False,
    experimental_io_device='/job:localhost',
    experimental_skip_checkpoint=False
)

tf_model = tf.keras.models.load_model(
    os.getcwd() + '/../../models/USE_model/'
)

In [2]:
def get_svm_classifier():
    return SVC(kernel='rbf', gamma='auto', class_weight='balanced', probability=True, random_state=122)

def get_rf_classifier():
    return RandomForestClassifier(n_estimators=300, random_state=122)

def get_lr_classifier():
    return LogisticRegression(class_weight='balanced', random_state=122)

def get_three_class_models():
    
    model_1 = get_svm_classifier()
    model_2 = get_svm_classifier()
    model_3 = get_lr_classifier()
    
    return [model_1, model_2, model_3]

def get_modified_vectors(vec_data):
    
    new_data = []
    for val in vec_data:
        new_data.append(val)
    
    new_data = np.array(new_data).reshape(-1, 512)
    return new_data

In [56]:
def get_threshold_output(preds, threshold):
    
    y_preds = []
    for val in preds:
        if val >= threshold:
            y_preds.append(1)
        else:
            y_preds.append(0)
        
    y_preds = np.array(y_preds)
    
    return y_preds

def get_f1_score_binary(y, preds, threshold, print_report=False):
    
    y_preds = get_threshold_output(preds, threshold)
    if print_report:
        print(classification_report(y, y_preds))
    
    metrics = precision_recall_fscore_support(y, y_preds)
    f1_score_tech = metrics[2][1]
    
    return f1_score_tech

def get_best_threshold(y, preds):
    
    threshold_vals = np.arange(0.1, 1, 0.001)
    f1_score_list = []
    
    for val in threshold_vals:
        f1_score_list.append(get_f1_score_binary(y, preds, val))

    max_idx = np.nanargmax(f1_score_list)
    thre_max = threshold_vals[max_idx]
    fscore = f1_score_list[max_idx]
    
    print(fscore)
    print(thre_max)
    
    return thre_max    

def get_trained_model_binary(X, y):
    
#     skf_f1score = perform_cross_validation(X, y, fold_cnt=5)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=123)
    
    model = get_svm_classifier()
    clf = make_pipeline(StandardScaler(with_mean=False), model)
    clf.fit(X_train, y_train)
    
    pred_probs = clf.predict_proba(X_test)[:,1]
    threshold = get_best_threshold(y_test, pred_probs)
    
    skf_f1score = 0
    
    return clf, skf_f1score, threshold

def get_test_f1score_binary(model, X_test, y_test, threshold):
    
    preds = model.predict_proba(X_test)
    preds = preds[:,1]

    f1_score = get_f1_score_binary(y_test, preds, threshold, print_report=True)
    
    return f1_score

In [24]:
def get_multi_class_metrics(y_test, preds, pr_flag=False):
    
    metrics = precision_recall_fscore_support(y_test, preds)
    
    f1_score_tech = metrics[2][1]
    f1_score_milt = metrics[2][2]
#     f1_score_milt = metrics[2][1]
    
    f1_score = (f1_score_tech+f1_score_milt)/2
    
    if pr_flag:
        print(classification_report(y_test, preds))
        return f1_score, preds
    
    return f1_score

def perform_cross_validation(X, y, fold_cnt=5):
    
    skf = StratifiedKFold(n_splits=fold_cnt, shuffle=True, random_state=123)
    f1_scores_list = []
    
    for train_idx, test_idx in skf.split(X, y):
        
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        model = get_svm_classifier()
        clf = make_pipeline(StandardScaler(with_mean=False), model)
        
        clf.fit(X_train, y_train)
        preds = clf.predict(X_test)

        f1_score = get_multi_class_metrics(y_test, preds, pr_flag=False)
        f1_scores_list.append(f1_score)
    
    return sum(f1_scores_list)/fold_cnt

def get_trained_model(X, y):
    
    skf_f1score = perform_cross_validation(X, y, fold_cnt=5)
    
    model = get_svm_classifier()
    clf = make_pipeline(StandardScaler(with_mean=False), model)
    clf.fit(X, y)
    
    return clf, skf_f1score

def get_test_f1score(model, X_test, y_test):
    
    preds = model.predict(X_test)
    f1_score, preds = get_multi_class_metrics(y_test, preds, pr_flag=True)
    
    return f1_score, preds

def get_performance_metrics(X_train, y_train, X_test, y_test, binary=False):
    
    if binary:
        model, cv_score, threshold = get_trained_model_binary(X_train, y_train)
        test_f1_score = get_test_f1score_binary(model, X_test, y_test,threshold)
        
        return model, threshold
    else:    
        model, cv_score = get_trained_model(X_train, y_train)
        test_f1_score, preds = get_test_f1score(model, X_test, y_test)
        
        test_df['pred_label'] = preds

        
        return model

    print()
    print(f'Training CV f1 score: {cv_score}')
    print(f'Test F1-score: {test_f1_score}')

In [5]:
train_df = pd.read_pickle(os.getcwd() + '/../dataframes/train_df_features.pkl')  ## train_df
test_df = pd.read_pickle(os.getcwd() + '/../dataframes/test_df_features.pkl') ## test_df

train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [7]:
unlabeled_df = pd.read_pickle(os.getcwd() + '/../dataframes/unlabeled_df_features.pkl') # unlabeled_df
# unlabeled_df['lang'] = unlabeled_df.apply(lambda x:detect_language(x['text'].revplace("\n"," ")), axis=1)

In [51]:
unlabeled_df

,id,text,doc_vec,lang,text_tokens,nc_vec
0,210705_news_466581.txt,"Whilst residing in Paris in the sixth century,...","[[0.0117851915, -0.013124656, 0.006891161, 0.0...",en,"([Paris, the sixth century, the widow, the mer...","[[-0.0061701364, -0.012984723, -0.02264499, -0..."
1,210705_news_466582.txt,"Early in 2018, the volcano Anak Krakatau in In...","[[-0.011649534, -0.033309218, -0.004555769, 0....",en,"([the volcano Anak Krakatau, Indonesia, it, a ...","[[-0.0047265957, -0.011267428, -0.01635613, -0..."
2,210705_news_466583.txt,"Posted by Cassidy Curtis, Visual Designer and ...","[[-0.07128145, 0.011230298, 0.045890186, -0.07...",en,"([Cassidy Curtis, Visual Designer, David Sales...","[[-0.005185811, -0.0008352126, -0.010343001, -..."
3,210705_news_466584.txt,One thing I love about living in Germany is th...,"[[-0.012562483, -0.03460731, 0.089639775, 0.02...",en,"([one thing, I, Germany, the bread, American W...","[[0.005709256, -0.015197817, -0.002575722, -0...."
4,210705_news_466585.txt,Embrace the Grind There’s this card trick I sa...,"[[-0.04854836, 0.07415119, 0.044061035, -0.008...",en,"([the Grind, this card trick, I, I, all the ti...","[[0.0018794591, -0.0019065727, -0.024219586, -..."
...,...,...,...,...,...,...
1805,210705_news_469717.txt,Und weshalb sollte die Union jetzt auf den stä...,"[[-0.032125708, 0.005915861, 0.03515286, 0.006...",de,"([der Union, der stark Kandidat, der Furcht, d...","[[-0.004311784, -0.00028916707, -0.020514684, ..."
1806,210705_news_469718.txt,Ars Technica Today's Dealmaster includes a r...,"[[0.045911647, 0.0021237172, -0.056985255, 0.0...",en,"([Ars Technica, today's Dealmaster, a return, ...","[[-0.013846245, 0.010529164, -0.007516246, 0.0..."
1807,210705_news_469719.txt,"Enlarge / A Yemeni khamsiyat (top left), a ...","[[0.027108321, -0.004789675, -0.036533337, 0.0...",en,"([a yemeni khamsiyat, a spanish real, (top rig...","[[0.008365323, -0.019968145, -0.02338454, -0.0..."
1808,210705_news_469722.txt,"Bridgerton , the raunchy Regency costume drama...","[[-0.0044380925, 0.0010670775, 0.06688552, 0.0...",en,"([Bridgerton, the raunchy Regency costume dram...","[[-0.011590819, -0.006641924, -0.014274846, -0..."


In [39]:
X_unlabeled = get_modified_vectors(unlabeled_df.nc_vec.values)

In [17]:
multi_model = get_performance_metrics(X_train_nc, y_train, X_test_nc, y_test)

              precision    recall  f1-score   support

           0       0.80      0.67      0.73       113
           1       0.38      0.19      0.25        16
           2       0.49      0.83      0.61        23
           3       0.35      0.45      0.39        33

    accuracy                           0.61       185
   macro avg       0.50      0.54      0.50       185
weighted avg       0.64      0.61      0.61       185



In [54]:
unlabeled_df['milt_label'] = multi_model.predict(X_unlabeled)

In [ ]:
test_df_new = test_df[test_df['pred_label'].isin([0,1,3])]

In [8]:
X_train_use = get_modified_vectors(train_df.doc_vec.values)
X_test_use = get_modified_vectors(test_df.doc_vec.values)

y_train = train_df.label.values
y_test = test_df.label.values

# y_train_new = np.array([val if val!=3 else 0 for val in y_train]).astype('int32')
# y_test_new = np.array([val if val!=3 else 0 for val in y_test]).astype('int32')

# y_train_tech = np.array([0 if val!=1 else 1 for val in y_train]).astype('int32')
# y_test_tech = np.array([0 if val!=1 else 1 for val in y_test]).astype('int32')

In [55]:
unlabeled_df_new = unlabeled_df[unlabeled_df['milt_label'].isin([0,1,3])]
X_unlabeled_new = get_modified_vectors(unlabeled_df_new.nc_vec.values)

In [59]:
preds = bin_model.predict_proba(X_unlabeled_new)[:,1]
unlabeled_df_new['tech_label'] = get_threshold_output(preds, threshold)

In [61]:
unlabeled_df['milt_label'].value_counts()

0    1596
3     176
1      20
2      17
Name: milt_label, dtype: int64

In [60]:
sum(unlabeled_df_new['tech_label'])

129

In [67]:
def write_document_data(data, filepath):

    with open(filepath, 'w') as f:
        json.dump(data, f)

In [68]:
tech_data_dict = dict()

for idx, row in unlabeled_df_new.iterrows():
    
    if row['tech_label'] == 1:
        tech_data_dict[row['id']] = {
            'page_id': row['id'],
            'text': row['text']
        }

In [69]:
milt_data_dict = dict()

for idx, row in unlabeled_df.iterrows():
    
    if row['milt_label'] == 2:
        milt_data_dict[row['id']] = {
            'page_id': row['id'],
            'text': row['text']
        }

In [70]:
write_document_data(tech_data_dict, os.getcwd()+'/../json_data/technologie_document_data.json')
write_document_data(milt_data_dict, os.getcwd()+'/../json_data/military_document_data.json')

In [22]:
y_train_tech = np.array([0 if val!=1 else 1 for val in train_df.label.values]).astype('int32')

X_test_new_nc = get_modified_vectors(test_df_new.nc_vec.values)
y_test_new_tech = np.array([0 if val!=1 else 1 for val in test_df_new.label.values]).astype('int32')

In [385]:
X_test_new_adj = get_modified_vectors(test_df_new.adj_vec.values)

In [25]:
bin_model, threshold = get_performance_metrics(X_train_nc, y_train_tech, X_test_new_nc, y_test_new_tech, binary=True)

0.25
0.10900000000000001
              precision    recall  f1-score   support

           0       0.97      0.79      0.87       134
           1       0.24      0.75      0.37        12

    accuracy                           0.79       146
   macro avg       0.61      0.77      0.62       146
weighted avg       0.91      0.79      0.83       146



In [366]:
get_performance_metrics(X_train_new_nc, y_train_new_tech, X_test_new_nc, y_test_new_tech, binary=True)

0.3
0.10500000000000001
              precision    recall  f1-score   support

           0       0.95      0.78      0.86       134
           1       0.19      0.58      0.29        12

    accuracy                           0.76       146
   macro avg       0.57      0.68      0.57       146
weighted avg       0.89      0.76      0.81       146


Training CV f1 score: 0
Test F1-score: 0.28571428571428575


In [328]:
get_performance_metrics(X_train_topic_use, y_train, X_test_topic_use, y_test_tech, binary=True)

0.3529411764705882
0.13100000000000003
              precision    recall  f1-score   support

           0       0.92      0.85      0.88       169
           1       0.13      0.25      0.17        16

    accuracy                           0.79       185
   macro avg       0.53      0.55      0.53       185
weighted avg       0.85      0.79      0.82       185


Training CV f1 score: 0
Test F1-score: 0.1739130434782609


In [26]:
get_performance_metrics(X_train_use, y_train, X_test_use, y_test)

              precision    recall  f1-score   support

           0       0.81      0.73      0.77       113
           1       0.43      0.19      0.26        16
           2       0.45      0.78      0.57        23
           3       0.38      0.42      0.40        33

    accuracy                           0.63       185
   macro avg       0.52      0.53      0.50       185
weighted avg       0.66      0.63      0.63       185


Training CV f1 score: 0.41471200104976197
Test F1-score: 0.4161490683229813


### 2. Topic features testing

In [9]:
def get_use_topic_score(doc_vec, topic):
    
    return cosine_similarity(doc_vec, topic_embeddings_dict[topic])[0][0]

In [324]:
topic_list = ['Wirtschaft und Finanzen', 'Bildung', 'Politik', 'Tierreich', 'Rechtswissenschaften und Rechtsprechung', 'Gesundheit', 'Automobilbranche', 'Unterhaltung', 'Sport', 'Werbung', 'Technologie', 'Innovation', 'Militär', 'Quantencomputer', 'Swarm', 'Architecture', 'Forschnung', 'Drone', 'Autonomous', 'Modernisierung', 'Prototype', 'efficiency', 'Notebook', 'Angriff', 'Smartphone', 'Corona', 'Hacking', 'Kunden', 'Robot', 'Künstliche Intelligenz', 'smart', 'algorithmus', 'sensor', 'energy', 'digitalen', 'attack']
# topic_list = ['Werbung', 'Technologie', 'Innovation', 'Militär', 'Quantencomputer', 'Swarm', 'Architecture', 'Forschnung', 'Drone', 'Autonomous', 'Modernisierung', 'Prototype', 'efficiency', 'Notebook', 'Angriff', 'Smartphone', 'Corona', 'Hacking', 'Kunden', 'Robot', 'Künstliche Intelligenz', 'smart', 'algorithmus', 'sensor', 'energy', 'digitalen', 'attack']

topic_embeddings_dict = dict()
for topic in topic_list:
    topic_embeddings_dict[topic] = tf_model(topic)['outputs'].numpy()[0].reshape(1, -1)

In [325]:
topic_col_list = []

for topic in topic_list:
    topic_col_name = topic.lower().replace(' ', '_') + '_sim'
    topic_col_list.append(topic_col_name)
    
    train_df[topic_col_name] = train_df.apply(lambda x:get_use_topic_score(x['doc_vec'], topic), axis=1)
    test_df[topic_col_name] = test_df.apply(lambda x:get_use_topic_score(x['doc_vec'], topic), axis=1)

In [326]:
X_train_topic_use = train_df[topic_col_list].values
X_test_topic_use = test_df[topic_col_list].values

In [249]:
get_performance_metrics(X_train_topic_use, y_train, X_test_topic_use, y_test)

              precision    recall  f1-score   support

           0       0.81      0.60      0.69       113
           1       0.10      0.12      0.11        16
           2       0.39      0.61      0.47        23
           3       0.30      0.39      0.34        33

    accuracy                           0.52       185
   macro avg       0.40      0.43      0.40       185
weighted avg       0.60      0.52      0.55       185


Training CV f1 score: 0.348627069009422
Test F1-score: 0.29134218964727443


### 3. Noun-chunks, Verb and Adjective

In [236]:
nlp_de = spacy.load("de_core_news_sm")
nlp_en = spacy.load("en_core_web_sm")

In [240]:
def get_document_data(text, lang):
    
    doc = None
    
    if lang == 'en':
        doc = nlp_en(text)
    elif lang == 'de':
        doc = nlp_de(text)
    elif lang == 'ko':
        return None
    
    noun_phrases_list = []
    verbs_list = []
    adjs_list = []
    
    for nc in doc.noun_chunks:
        noun_phrases_list.append(nc.lemma_) 
        
    for token in doc:
        if token.pos_ == "ADJ":
            adjs_list.append(token.lemma_)
        elif token.pos_ == "VERB":
            verbs_list.append(token.lemma_)
            
    return (noun_phrases_list, verbs_list, adjs_list)

def get_avg_token_vector(token_list):
    
    avg_token_vec = []
    for token in token_list:
        avg_token_vec.append(tf_model(token)['outputs'].numpy()[0].reshape(1, -1))
        
    return np.mean(avg_token_vec, axis=0)

def get_mean_vector(vec_list):
    return np.mean(vec_list, axis=0)

In [244]:
unlabeled_df.lang.value_counts()

en    1284
de     525
Name: lang, dtype: int64

In [241]:
unlabeled_df['text_tokens'] = unlabeled_df.apply(lambda x:get_document_data(x['text'], x['lang']), axis=1)

In [243]:
unlabeled_df = unlabeled_df[unlabeled_df['lang'].isin(['en', 'de'])]

In [245]:
unlabeled_df.to_pickle(os.getcwd() + '/../dataframes/unlabeled_df_features.pkl')

In [15]:
# train_df['text_tokens'] = train_df.apply(lambda x:get_document_data(x['text'], x['lang']), axis=1)
# test_df['text_tokens'] = test_df.apply(lambda x:get_document_data(x['text'], x['lang']), axis=1)

In [16]:
# train_df['nc_vec'] = train_df.apply(lambda x:get_avg_token_vector(x['text_tokens'][0]), axis=1)
# train_df['verb_vec'] = train_df.apply(lambda x:get_avg_token_vector(x['text_tokens'][1]), axis=1)
# train_df['adj_vec'] = train_df.apply(lambda x:get_avg_token_vector(x['text_tokens'][2]), axis=1)

# test_df['nc_vec'] = test_df.apply(lambda x:get_avg_token_vector(x['text_tokens'][0]), axis=1)
# test_df['verb_vec'] = test_df.apply(lambda x:get_avg_token_vector(x['text_tokens'][1]), axis=1)
# test_df['adj_vec'] = test_df.apply(lambda x:get_avg_token_vector(x['text_tokens'][2]), axis=1)

In [61]:
# train_df.to_pickle(os.getcwd() + '/../dataframes/train_df_features.pkl')
# test_df.to_pickle(os.getcwd() + '/../dataframes/test_df_features.pkl')

In [9]:
X_train_nc = get_modified_vectors(train_df.nc_vec.values)
X_test_nc = get_modified_vectors(test_df.nc_vec.values)

X_train_verb = get_modified_vectors(train_df.verb_vec.values)
X_test_verb = get_modified_vectors(test_df.verb_vec.values)

X_train_adj = get_modified_vectors(train_df.adj_vec.values)
X_test_adj = get_modified_vectors(test_df.adj_vec.values)

In [34]:
def get_features_concatenate(vecs_1, vecs_2):
    return np.concatenate((vecs_1,vecs_2), axis=1)

def get_features_mean(vecs_1, vecs_2):
    
    return np.mean( (vecs_1,vecs_2), axis=0 )

In [35]:
X_train_nc_ad = get_features_mean(X_train_nc, X_train_adj)
X_test_nc_ad = get_features_mean(X_test_nc, X_test_adj)

In [36]:
get_performance_metrics(X_train_nc_ad, y_train, X_test_nc_ad, y_test)

              precision    recall  f1-score   support

           0       0.82      0.63      0.71       113
           1       0.30      0.19      0.23        16
           2       0.45      0.83      0.58        23
           3       0.35      0.48      0.41        33

    accuracy                           0.59       185
   macro avg       0.48      0.53      0.48       185
weighted avg       0.64      0.59      0.60       185


Training CV f1 score: 0.42320817864428406
Test F1-score: 0.40769230769230763


In [10]:
get_performance_metrics(X_train_nc, y_train, X_test_nc, y_test)

              precision    recall  f1-score   support

           0       0.80      0.67      0.73       113
           1       0.38      0.19      0.25        16
           2       0.49      0.83      0.61        23
           3       0.35      0.45      0.39        33

    accuracy                           0.61       185
   macro avg       0.50      0.54      0.50       185
weighted avg       0.64      0.61      0.61       185


Training CV f1 score: 0.3812573047867166
Test F1-score: 0.4314516129032258


In [28]:
get_performance_metrics(X_train_verb, y_train, X_test_verb, y_test)

              precision    recall  f1-score   support

           0       0.81      0.76      0.79       113
           1       0.40      0.25      0.31        16
           2       0.45      0.65      0.54        23
           3       0.25      0.27      0.26        33

    accuracy                           0.62       185
   macro avg       0.48      0.48      0.47       185
weighted avg       0.63      0.62      0.62       185


Training CV f1 score: 0.2532680007508382
Test F1-score: 0.4217032967032967


In [29]:
get_performance_metrics(X_train_adj, y_train, X_test_adj, y_test)

              precision    recall  f1-score   support

           0       0.75      0.68      0.71       113
           1       0.36      0.31      0.33        16
           2       0.48      0.61      0.54        23
           3       0.36      0.42      0.39        33

    accuracy                           0.59       185
   macro avg       0.49      0.51      0.49       185
weighted avg       0.61      0.59      0.60       185


Training CV f1 score: 0.26448179271708677
Test F1-score: 0.4358974358974359


### 4. Combined models

In [31]:
def create_stage_one_models(X_1, X_2, y, model_1, model_2):
    
    model_1 = make_pipeline(StandardScaler(with_mean=False), model_1)
    model_1.fit(X_1, y)
    
    model_2 = make_pipeline(StandardScaler(with_mean=False), model_2)
    model_2.fit(X_2, y)  
    
    return model_1, model_2

def create_stage_two_model(X, y, model_3):
    
    model_3 = make_pipeline(StandardScaler(with_mean=False), model_3)
    model_3.fit(X, y) 
    
    return model_3

def get_transformed_features(model_1, model_2, X_1, X_2, y, y_flag=False):
    
    preds_1 = model_1.predict_proba(X_1)
    preds_2 = model_2.predict_proba(X_2)
    
    combined_features = get_features_concatenate(preds_1, preds_2)
    if not y_flag:
        return combined_features
    
    return combined_features, y

def get_finalmodel_pipeline(models, X_train_1, X_train_2, y_train, y_flag=False):
    
    model_1, model_2 = create_stage_one_models(X_train_1, X_train_2, y_train, models[0], models[1])
    X, y = get_transformed_features(model_1, model_2, X_train_1, X_train_2, y_train, y_flag=True)
    
    model_3 = create_stage_two_model(X, y, models[2])
    
    return [model_1, model_2, model_3]

def get_test_f1score_combined(models, X_test_1, X_test_2, y_test):
    
    X_test = get_transformed_features(models[0], models[1], X_test_1, X_test_2, y_test, y_flag=False)
    preds = models[2].predict(X_test)
    
    f1_score = get_multi_class_metrics(y_test, preds, pr_flag=True)
    
    return f1_score

In [186]:
def perform_cross_validation_combined(X_1, X_2, y, fold_cnt=5):
    
    skf = StratifiedKFold(n_splits=fold_cnt, shuffle=True, random_state=123)
    f1_scores_list = []
    
    for train_idx, test_idx in skf.split(X_1, y):
        
        X_train_1, X_train_2, X_test_1, X_test_2 = X_1[train_idx], X_2[train_idx], X_1[test_idx], X_2[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        models = get_three_class_models()
        models = get_finalmodel_pipeline(models, X_train_1, X_train_2, y_train)
        
        X_test = get_transformed_features(models[0], models[1], X_test_1, X_test_2, y_test, y_flag=False)
        preds = models[2].predict(X_test)

        f1_score = get_multi_class_metrics(y_test, preds, pr_flag=False)
        f1_scores_list.append(f1_score)
    
    return sum(f1_scores_list)/fold_cnt

def get_trained_model_combined(X_1, X_2, y):
    
    skf_f1score = perform_cross_validation_combined(X_1, X_2, y, fold_cnt=5)
    
    models = get_three_class_models()
    models = get_finalmodel_pipeline(models, X_1, X_2, y)

    return models, skf_f1score
    

def get_performance_metrics_combined(X_train_1, X_train_2, y_train, X_test_1, X_test_2, y_test):
    
    models, cv_score = get_trained_model_combined(X_train_1, X_train_2, y_train)
    test_f1_score = get_test_f1score_combined(models, X_test_1, X_test_2, y_test)

    print()
    print(f'Training CV f1 score: {cv_score}')
    print(f'Test F1-score: {test_f1_score}')

In [207]:
get_performance_metrics_combined(X_train_nc, X_train_verb, y_train, X_test_nc, X_test_verb, y_test)

              precision    recall  f1-score   support

           0       0.69      0.87      0.77       113
           1       1.00      0.12      0.22        16
           2       0.44      0.17      0.25        23
           3       0.38      0.36      0.37        33

    accuracy                           0.63       185
   macro avg       0.63      0.38      0.40       185
weighted avg       0.63      0.63      0.59       185


Training CV f1 score: 0.24217057796005168
Test F1-score: 0.2361111111111111


In [210]:
get_performance_metrics_combined(X_train_nc, X_train_adj, y_train, X_test_nc, X_test_adj, y_test)

              precision    recall  f1-score   support

           0       0.67      0.88      0.76       113
           1       1.00      0.06      0.12        16
           2       0.60      0.26      0.36        23
           3       0.33      0.24      0.28        33

    accuracy                           0.62       185
   macro avg       0.65      0.36      0.38       185
weighted avg       0.63      0.62      0.57       185


Training CV f1 score: 0.31169002050580996
Test F1-score: 0.24064171122994654


In [209]:
get_performance_metrics_combined(X_train_nc, X_train_use, y_train, X_test_nc, X_test_use, y_test)

              precision    recall  f1-score   support

           0       0.69      0.89      0.78       113
           1       1.00      0.12      0.22        16
           2       0.50      0.30      0.38        23
           3       0.39      0.27      0.32        33

    accuracy                           0.64       185
   macro avg       0.65      0.40      0.43       185
weighted avg       0.64      0.64      0.60       185


Training CV f1 score: 0.3365809354044648
Test F1-score: 0.3003003003003003
